In [1]:
import pandas as pd

# Load the ground truth and mapping table
ground_truth_path = "ground_truth.xlsx"
mapping_table_path = "Manual_Mapping_Table.xlsx"
ground_truth = pd.read_excel(ground_truth_path)
manual_mapping_table = pd.read_excel(mapping_table_path)

# Load the model outputs
dsv3_output = pd.read_csv("DeepSeekV3_raw_Results.csv")


# Normalize data for consistency
manual_mapping_table["Extracted Attribute"] = manual_mapping_table["Extracted Attribute"].str.lower().str.strip()
manual_mapping_table["Mapped Attribute"] = manual_mapping_table["Mapped Attribute"].str.lower().str.strip()
manual_mapping_table["Category"] = manual_mapping_table["Category"].str.lower().str.strip()

ground_truth["Attribute"] = ground_truth["Attribute"].str.lower().str.strip()
ground_truth["Category"] = ground_truth["Category"].str.lower().str.strip()

for output in [dsv3_output]:
    output["Attribute"] = output["Attribute"].str.lower().str.strip()
    output["Category"] = output["Category"].str.lower().str.strip()

# Apply manual mapping to model outputs
def apply_mapping(model_output, mapping_table):
    """
    Map extracted attributes in model output to ground truth attributes using the consolidated mapping table.
    """
    # Merge model output with the mapping table
    mapped_output = model_output.merge(
        mapping_table,
        left_on=["Category", "Attribute"],
        right_on=["Category", "Extracted Attribute"],
        how="left"
    )
    
    # Use the 'Mapped Attribute' if matched, or retain the original attribute if not
    mapped_output["Mapped Attribute"] = mapped_output["Mapped Attribute"].fillna(mapped_output["Attribute"])
    
    # Keep only relevant columns
    mapped_output = mapped_output[["Category", "Mapped Attribute", "Count", "Percentage"]]

    return mapped_output

# Apply the mapping function to each model output
mapped_dsv3 = apply_mapping(dsv3_output, manual_mapping_table)


# Calculate evaluation metrics
def calculate_metrics(mapped_output, ground_truth):
    """
    Calculate Precision, Recall, F1-Score, and Coverage based on mapped attributes.
    """
    # True positives
    true_positives = mapped_output[mapped_output["Mapped Attribute"].isin(ground_truth["Attribute"])]
    
    # False positives
    false_positives = mapped_output[~mapped_output["Mapped Attribute"].isin(ground_truth["Attribute"])]
    
    # False negatives
    false_negatives = ground_truth[~ground_truth["Attribute"].isin(mapped_output["Mapped Attribute"])]

    # Calculate metrics
    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if len(true_positives) + len(false_positives) > 0 else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if len(true_positives) + len(false_negatives) > 0 else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0
    coverage = mapped_output["Mapped Attribute"].nunique() / ground_truth["Attribute"].nunique()

    return precision, recall, f1_score, coverage

# Evaluate metrics for each model
metrics_dsv3 = calculate_metrics(mapped_dsv3, ground_truth)


# Print results
print(f"dsv3 Metrics: Precision: {metrics_dsv3[0]:.2f}, Recall: {metrics_dsv3[1]:.2f}, F1-Score: {metrics_dsv3[2]:.2f}, Coverage: {metrics_dsv3[3]:.2f}")


dsv3 Metrics: Precision: 0.16, Recall: 0.80, F1-Score: 0.27, Coverage: 8.21


In [2]:
mapped_dsv3.to_csv('mapped_dsv3.csv')